In [ ]:
from IPython.display import display


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
import csv
import math
from collections import defaultdict

data = defaultdict(list)
with open("evolution_directions_fermes_local.csv", "r") as data_f, open("regressions_local.csv", "r") as reg_data_f:
    reg_data = defaultdict(dict)
    for f in list(csv.DictReader(reg_data_f)):
        reg_data[f['direction_ferme']][f['kind']] = f
    for d in csv.DictReader(data_f):
        data[d['direction_ferme']].append({
            "year": d['year'],
            "value": float(d['value']), 
            "kind" : d['kind'],
            "reg_point": math.exp(float(reg_data[d['direction_ferme']][d['kind']]['slope'])*float(d['year'])+float(reg_data[d['direction_ferme']][d['kind']]['intercept'])) ,
            "direction_ferme": d['direction_ferme']})



In [ ]:
data

In [ ]:
for direction,values in data.items():
  VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "data": {"values": values},
    "facet":{
      "column" : {"field": "kind"}
    },
    "spec":{
    "layer": [
      {
        "mark": {
          "type": "point",
          "filled": True
        },
        "encoding": {
          "x": {
            "field": "year",
            "type": "temporal",
            "scale": {"domain": ["1740","1789"]}
          },
          "y": {
            "field": "value",
            "type": "quantitative"
          }
        }
      },
      {
        "mark": {
          "type": "line",
          "color": "firebrick"
        },
        "encoding": {
          "x": {
            "field": "year",
            "type": "temporal",
            "scale": {"domain": ["1750","1780"]}
          },
          "y": {
            "field": "reg_point",
            "type": "quantitative"
          }
        }
      },
      {
      
      "mark": {
        "type": "text",
        "color": "firebrick",
        "x": "width",
        "align": "right",
        "y": -5
      },
      "encoding": {
        "text": {"type": "nominal", "field": "direction_ferme"}
      }
    }
    ]}
  })